In [ ]:
# create a seperate folder to store everything
!mkdir training
%cd training

/content/training


In [ ]:
# clone the repo for running finetuning
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 697 (delta 278), reused 344 (delta 240), pack-reused 297
Receiving objects: 100% (697/697), 2.64 MiB | 10.12 MiB/s, done.
Resolving deltas: 100% (405/405), done.
/content/training/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1399, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 1399 (delta 135), reused 144 (delta 117), pack-reused 1219
Receiving objects: 100% (1399/1399), 9.57 MiB | 9.27 MiB/s, done.
Resolving deltas: 100% (745/745), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 1

In [ ]:
! sudo apt install tree

# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (207 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120880 files and directories currently installe

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
This is just a warning: triton is not available


In [ ]:
## for the latest samanantar dataset v0.3 -> please use this link: https://storage.googleapis.com/samanantar-public/V0.3/source_wise_splits.zip
# This v0.3 dataset has source wise splits to indicate where the data has been collected from
# For preprocessing simplicity we will use v0.2( which just uses raw text files without source information) in this tutorial
#
#
#  lets now download the indictrans data v0.2 dataset
! wget https://storage.googleapis.com/samanantar-public/V0.2/data/en2indic/samanatar-en-indic-v0.2.zip




# training data is organized as en-X folders where each folder contains two text files containing parallel data for en-X lang pair.

# final_data
# ├── en-as
# │   ├── train.as
# │   └── train.en
# ├── en-bn
# │   ├── train.bn
# │   └── train.en
# ├── en-gu
# │   ├── train.en
# │   └── train.gu
# ├── en-hi
# │   ├── train.en
# │   └── train.hi
# ├── en-kn
# │   ├── train.en
# │   └── train.kn
# ├── en-ml
# │   ├── train.en
# │   └── train.ml
# ├── en-mr
# │   ├── train.en
# │   └── train.mr
# ├── en-or
# │   ├── train.en
# │   └── train.or
# ├── en-pa
# │   ├── train.en
# │   └── train.pa
# ├── en-ta
# │   ├── train.en
# │   └── train.ta
# └── en-te
#     ├── train.en
#     └── train.te


! unzip samanatar-en-indic-v0.2.zip



--2023-11-18 09:15:14--  https://storage.googleapis.com/samanantar-public/V0.2/data/en2indic/samanatar-en-indic-v0.2.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.207, 142.250.136.207, 142.250.148.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3376507188 (3.1G) [application/zip]
Saving to: ‘samanatar-en-indic-v0.2.zip’

samanatar-en-indic- 100%[===================>]   3.14G   114MB/s    in 31s     

2023-11-18 09:15:45 (103 MB/s) - ‘samanatar-en-indic-v0.2.zip’ saved [3376507188/3376507188]

Archive:  samanatar-en-indic-v0.2.zip
   creating: final_data/
   creating: final_data/en-kn/
  inflating: final_data/en-kn/train.kn  
  inflating: final_data/en-kn/train.en  
   creating: final_data/en-bn/
  inflating: final_data/en-bn/train.en  
  inflating: final_data/en-bn/train.bn  
   creating: final_data/en-as/
  inflating: final_data/en-as/train.as

In [ ]:
# benchmarks folder consists of all the benchmarks we report in the paper - pmi, ufal-ta, wat2020, wat2021, wmt-news

! unzip benchmarks.zip

Archive:  benchmarks.zip
   creating: benchmarks/
   creating: benchmarks/pmi/
   creating: benchmarks/pmi/en-as/
  inflating: benchmarks/pmi/en-as/dev.as  
  inflating: benchmarks/pmi/en-as/dev.en  
  inflating: benchmarks/pmi/en-as/test.as  
  inflating: benchmarks/pmi/en-as/test.en  
   creating: benchmarks/wat2021-devtest/
  inflating: benchmarks/wat2021-devtest/dev.gu  
  inflating: benchmarks/wat2021-devtest/dev.en  
  inflating: benchmarks/wat2021-devtest/test.bn  
  inflating: benchmarks/wat2021-devtest/dev.bn  
  inflating: benchmarks/wat2021-devtest/test.hi  
  inflating: benchmarks/wat2021-devtest/dev.kn  
  inflating: benchmarks/wat2021-devtest/dev.ta  
  inflating: benchmarks/wat2021-devtest/test.pa  
  inflating: benchmarks/wat2021-devtest/test.en  
  inflating: benchmarks/wat2021-devtest/test.mr  
  inflating: benchmarks/wat2021-devtest/test.kn  
  inflating: benchmarks/wat2021-devtest/dev.ml  
  inflating: benchmarks/wat2021-devtest/test.ta  
  inflating: benchmarks/wat

In [ ]:
# prepare_data_joint_training.sh takes experiment dir, src_lang, tgt_lang as input
# This does preprocessing, building vocab, binarization for joint training

# The learning  and applying vocabulary will take a while if the dataset is huge. To make it faster, run it on a multicore system

! bash prepare_data_joint_training.sh '../indic-en-exp' 'indic' 'en'

Running experiment ../indic-en-exp on indic to en
Applying normalization and script conversion for train
100% 138353/138353 [00:17<00:00, 7981.73it/s]
100% 138353/138353 [00:33<00:00, 4161.55it/s]
Number of sentences in train: 138353
Applying normalization and script conversion for dev
Traceback (most recent call last):
  File "/content/training/indicTrans/scripts/preprocess_translate.py", line 172, in <module>
    print(preprocess(infname, outfname, lang, transliterate))
  File "/content/training/indicTrans/scripts/preprocess_translate.py", line 61, in preprocess
    num_lines = sum(1 for line in open(infname, "r"))
FileNotFoundError: [Errno 2] No such file or directory: '../indic-en-exp/devtest/all/en-as/dev.as'
Traceback (most recent call last):
  File "/content/training/indicTrans/scripts/preprocess_translate.py", line 172, in <module>
    print(preprocess(infname, outfname, lang, transliterate))
  File "/content/training/indicTrans/scripts/preprocess_translate.py", line 61, in pre

In [ ]:
# create an experiment dir to store train data, devtest data.
# This folder will also store vocabulary files (created with subword_nmt for bpe), fairseq bin files (for training), model checkpoints.

# for this example we will be training indic to en translation model. We will name our exp_dir as indic-en-exp
! mkdir indic-en-exp
# copying all the train folders to exp_dir
! cp -r final_data/* indic-en-exp

! mkdir -p indic-en-exp/devtest

# copying all benchmarks to devtest folder in exp_dir
! cp -r benchmarks/* indic-en-exp/devtest

# folder to store combined devtest data (based on the domains you want to test, you can combine multiple benchmarks dev datasets, remove duplicates)
! mkdir -p indic-en-exp/devtest/all

# in this tutorial, for simplicity, we will just use wat2020 devtest for dev and test set
! cp -r indic-en-exp/devtest/wat2020-devtest/* indic-en-exp/devtest/all



In [ ]:
%cd indicTrans

/content/training/indicTrans


In [4]:
# Training the model

# pls refer to fairseq documentaion to know more about each of these options (https://fairseq.readthedocs.io/en/latest/command_line_tools.html)


# some notable args:
# --max-updates         -> maximum update steps the model will be trained for
# --arch=transformer_4x -> we use a custom transformer model and name it transformer_4x (4 times the parameter size of transformer  base)
# --user_dir            -> we define the custom transformer arch in model_configs folder and pass it as an argument to user_dir for fairseq to register this architechture
# --lr                  -> learning rate. From our limited experiments, we find that lower learning rates like 3e-5 works best for finetuning.
# --max_tokens          -> this is max tokens per batch. You should limit to lower values if you get oom errors.
# --update-freq         -> gradient accumulation steps


!( fairseq-train ../indic-en-exp/final_bin \
--max-source-positions=210 \
--max-target-positions=210 \
--max-update=<max_updates> \
--save-interval=1 \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--lr-scheduler=inverse_sqrt \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--lr 0.0005 \
--warmup-updates 4000 \
--dropout 0.2 \
--save-dir ../indic-en-exp/model \
--keep-last-epochs 5 \
--patience 5 \
--skip-invalid-size-inputs-valid-test \
--fp16 \
--user-dir model_configs \
--wandb-project <wandb_project_name> \
--update-freq=<grad_accumulation_steps> \
--distributed-world-size <num_gpus> \
--max-tokens <max_tokens_in_a_batch> )

In [5]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score

def calculate_scores(reference_translations, predicted_translations, language_pair):
    # Calculate BLEU score
    bleu_score = corpus_bleu(reference_translations, predicted_translations)

    # Calculate METEOR score
    meteor_score_val = meteor_score(reference_translations, predicted_translations)

    # Print the results
    print(f"{language_pair}:\nBLEU Score: {bleu_score:.2f}\nMETEOR Score: {meteor_score_val:.2f}\n")


# Calculate and print scores for each language pair
calculate_scores(english_to_french_references, english_to_french_predictions, "English to French")
calculate_scores(german_to_english_references, german_to_english_predictions, "German to English")
calculate_scores(chinese_to_english_references, chinese_to_english_predictions, "Chinese to English")
calculate_scores(spanish_to_portuguese_references, spanish_to_portuguese_predictions, "Spanish to Portuguese")
calculate_scores(japanese_to_korean_references, japanese_to_korean_predictions, "Japanese to Korean")


English to French:
BLEU Score: 0.30
METEOR Score: 0.50

German to English:
BLEU Score: 0.25
METEOR Score: 0.45

Chinese to English:
BLEU Score: 0.15
METEOR Score: 0.38

Spanish to Portuguese:
BLEU Score: 0.28
METEOR Score: 0.48

Japanese to Korean:
BLEU Score: 0.22
METEOR Score: 0.42
